# Edit rules for categorical data

## 1. Introduction

During the theory lectures, the theory behind edit rules is explained in detail.
During this assignment, you will use edit rules for categorical data in a practical fashion.
More specifically, you will use edit rules based on a small dataset that captures data about clinical trials.
This dataset is composed by combining attributes from two different sources, being [EudraCT](https://eudract.ema.europa.eu/) and [ClinicalTrials.gov](https://clinicaltrials.gov/).
You will have to detect inconsistencies between these sources by means of the Fellegi-Holt framework.
Later during this course, methods are introduced to fix these inconsistencies by means of imputation.

Before you can start with detecting inconsistencies, we will introduce the pandas library, which offers a variety of functions for processing and analyzing data, which will be used during the remainder of this assignment.


## 2. The pandas library

During this assignment, you will extensively use the [pandas](https://pandas.pydata.org/) library in Pyhton.
This library is mainly used for reading, processing and analyzing datasets.
To get familiar with this library, you will start with some pandas-based exercises.
By means of these exercises, you will also get familiar with the given dataset (clinical_trials.csv).

In [ ]:
import pandas as pd

In [ ]:
# Exercise: Read the 'clinical_trials.csv' dataset by means of the pandas library
dataset = #...

You can see that it is straightforward to read a .csv file by means of the pandas library.
When you do this, a pandas DataFrame object is created that represents a table with rows and attributes matching respectively the rows and columns of the given .csv file.
In the following, we will look into this DataFrame object in detail.

In [ ]:
# Exercise: count the number of rows and columns in the DataFrame object
number_of_rows = #...
print("Number of rows: {:d}".format(number_of_rows))

number_of_columns = #...
print("Number of columns: {:d}".format(number_of_columns))

To know which values and columns are persisted in the dataset, it is possible to look into a sample (first rows) of the dataset.
Besides that, it is also possible to retrieve, for each column separately, which values are persisted.

In [ ]:
# Exercise: print the first ten rows of the dataset
print("First ten rows of the dataset: ")
# ...

In [ ]:
# Exercise: print, for each column, the name of the column and a set of unique values that are persisted for this column
print("All columns and column values: ")
# ...

If you have managed to complete the previous exercise successfully, you can see that the dataset consists of 4 columns: 'open', 'double_blind', 'single_blind' and 'masking'.
The first three columns contain boolean values which are collected from the [EudraCT](https://eudract.ema.europa.eu/) dataset and the values in the column 'masking' are collected from the [ClinicalTrials.gov](https://clinicaltrials.gov/) dataset.
Each row specifies details related to the design of a clinical trial that is persisted in both datasets.

Before continuing, an overview is given of the semantical meaning of the different columns.

* **open**: a study in which both the participants and the researchers know which treatment is given;
* **single_blind**: a study in which only one of both parties (mostly the researchers) know which treatment is given;
* **double_blind**: a study in which none of the parties know which treatment is given;
* **masking**: categorical attribute containing values related to the masking/blindness of clinical trials (0 = Open, 1 = Single, 2 = Double, >2 = Triple/Quadruple).

Besides that, each column can contain a special value: NaN.
This value represents a `null`-value and indicates that certain data is not known.
The more `null`-values appear, the less accurate certain analysis based on this data will reflect the reality, which is obviously harmful for a researcher.
During the next exercises, you will assess how many column values and rows contain `null` values.

In [ ]:
# Exercise: count the total number of column values (cells) that contain a null-value
print("Number of null-values:")
# ...
# 5202

In [ ]:
# Exercise: count the total number of rows that contain a null-value
print("Number of rows containing a null-value:")
# ...
# 2083

In [ ]:
# Exercise: print, for each column separately, the total number of null-values
print("Number of null-values per attribute:")
# ...
# 1547,1622,1779,254

Finally, it is also possible to select certain columns and rows that satisfy given criteria in order to find data more easily.

In [ ]:
# Exercise: print, for the first 10 rows, the values of column 'open'
# ...

In [ ]:
# Exercise: print, for the first 10 rows, the values of columns 2 to 4
# ...

In [ ]:
# Exercise: print the content of the row with index 1000
# ...
# open            Yes
# double_blind     No
# single_blind     No
# masking           0

In [ ]:
# Exercise: print the content of the first 10 rows in which column 'single_blind' contains the value 'Yes'
# ...
# rijen 8, 15, 24, 30, 32, 33, 54, 62, 88, 126

In [ ]:
# Exercise: count the total number of rows in which column 'single_blind' contains the value 'Yes' and column 'masking' contains the value '1'
# ...
# 287

In [ ]:
# Exercise: print the content of the first 10 rows that do NOT contain null-values
# ...
# 1, 2, 3, 4, 5, 7, 8, 9, 11, 12

## 3. Edit rules

Now you have practiced with the [pandas](https://pandas.pydata.org/) library in Python and investigated the dataset that we are going to use during this assigment, it is time to assess the quality of this dataset and to enhance this quality by means of data quality rules.
As we mentioned before, the dataset captures clinical trial design data extracted from two different data sources.
In the remainder, you will investigate how consistent this data is within (intra-consistency) and between (inter-consistency) these two different sources.

The basic principle that we are going to apply to assess the quality of a dataset (or part of a dataset) by means of a rule-based approach is the following.
* First, a set of quality rules is constructed that must be satisfied by each object (row) in the dataset.
In our case, these rules define constraints on attribute values or on combinations of attribute values in the dataset.
* Satisfaction of each quality rule is tested by each data object and results in a boolean value ('yes, the object satisfies the rule' or 'no, the object does not satisfy the rule').
* The more rules that are satisfied by a data object, the higher the quality of the data object is.

### 3.1 Definition

In this section, we will fill in this basic principle by means of *edit rules* on data objects (rows) of the given dataset.
Edit rules are introduced by Fellegi & Holt in their research paper: A Systematic Approach to Automatic Edit and Imputation (1976) and form an elegant way to represent non-permissible combinations of values. 

The general definition of an edit rule is the following.
An *edit rule* over a dataset $R$ with attributes $a_1,\dots,a_k$ is a rule of type $E_1 \times \dots \times E_k$ in which each $E_i$ is a non-empty subset of $A_i$ (with $A_i$ the domain of attribute $a_i$, without `null`-values!).
An edit rule semantically defines a set of non-permissible rows within $A_1 \times \dots \times A_k$.
When a row is part of this set, it does **not** satisfy (or it fails) the edit rule.
In any other case, it does.

We already have defined a set of eight edit rules on our dataset.
These can be found in the following table.

| Rule  | open      | single_blind | double_blind | masking      |
|-------|-----------|--------------|--------------|--------------|
| 1     | Yes       | dom(single)  | Yes          | dom(masking) |
| 2     | dom(open) | Yes          | Yes          | dom(masking) |
| 3     | Yes       | Yes          | dom(double)  | dom(masking) |
| 4     | dom(open) | No           | dom(double)  | 1            |
| 5     | dom(open) | dom(single)  | No           | 2            |
| 6     | dom(open) | dom(single)  | Yes          | 0            |
| 7     | dom(open) | Yes          | dom(double)  | 0            |
| 8     | dom(open) | dom(single)  | Yes          | 1            |

As an example, we will investigate edit rule 1 ({Yes} $\times$ dom(single) $\times$ {Yes} $\times$ dom(masking)) together.
This rule states that *open* = 'Yes' and *double_blind* = 'Yes' may **not** appear together, independent of the value of the other two attributes (dom($a_i$) represents the entire domain of attribute $a_i$).
We also say that attributes *open* and *double_blind* enter in edit rule 1, because the value sets for these attributes are a real subset of the domain of the attributes.
Semantically, this edit rule makes sense, because it is not permitted that a study is both 'open' and 'double_blind'.
Before continuing, try to investigate the other edit rules by interpreting them semantically.
Also, list for each edit rule which attributes are entering.
Make sure to understand the definition of edit rules well!

In [ ]:
# Exercise: count the number of rows from the given dataset that fail edit rule 1 given in the table above.
failing_rows = # ...
print("Number of rows failing edit rule 1: {:d}".format(failing_rows))

In the following piece of code, we already initialized the given eight edit rules for you, which you will need during the remainder of the assignment.
If an attribute is not listed in an edit rule, it does not enter in the edit rule and has no influence on satisfaction of the rule.
Before continuing, think about why this last statement is true.

In [ ]:
edit_rules = [
                {
                    'open' : 'Yes',
                    'double_blind' : 'Yes'
                },
                {
                    'single_blind' : 'Yes',
                    'double_blind' : 'Yes'
                },
                {
                    'open' : 'Yes',
                    'single_blind' : 'Yes'
                }, 
                {
                    'single_blind' : 'No',
                    'masking' : '1'
                },
                {
                    'double_blind' : 'No',
                    'masking' : '2'
                },
                {
                    'double_blind' : 'Yes',
                    'masking' : '0'
                },
                {
                    'single_blind' : 'Yes',
                    'masking' : '0'
                },
                {
                    'double_blind' : 'Yes',
                    'masking' : '1'
                }
             ]

print("Edit rules: ")
for edit_rule in edit_rules:
    print(edit_rule)

In [ ]:
# Exercise: implement an algorithm that persists, per row number, the edit rules that are failed by the corresponding row in the violations dictionary-object.
# Do not persist rows that satisfy all edit rules.
# The edit rules may be represented by their index in the Python-list edit_rules

def get_violated_rows(dataset, edit_rules):
    violations = {}

    for index, row in dataset.iterrows():

        # debug info
        if index % 5000 == 0:
            print(str(index) + " rows processed")
            
        # TO IMPLEMENT...
            
    return violations

violations = get_violated_rows(dataset, edit_rules)
print("number of violated rows: {:d}".format(len(violations.keys())))

# 1222

In [ ]:
# Exercise: validate for the first row persisted in the violations dictionary whether your algorithm gives a correct result
row = next(iter(violations))
print(dataset.iloc[row])

for edit_rule_index in violations[row]:
    print(edit_rules[edit_rule_index])

### 3.2 Complete set van edit regels

In the previous section, we tested which rows in the dataset fail the given set of edit rules.
In this section, we will investigate which attributes potentially contain errors within these failing rows such that we can adapt this value later.

It is straightforward to see that a row can only satisfy a failing edit rule if the value of an attribute that enters in the edit rule is adapted.

As an example, take the following row.

| open      | single_blind | double_blind | masking      |
|-----------|--------------|--------------|--------------|
| Yes       | No           | Yes          | 2            |

This row violates edit rule 1: {Yes} $\times$ dom(single) $\times$ {Yes} $\times$ dom(masking) and attributes *open* and *double_blind* enter in this edit rule, because the value sets of these attributes are real subsets of their domains.
In order to make this row satisfy edit rule 1, it is necessary to adapt the value of attribute *open* or attribute *double_blind*.
Adaptation of the values of attributes *single_blind* or *masking* does not influence satisfaction of this edit rule.

If we choose to adapt the value of attribute 'open' to 'No' (the only remaining value within the domain), we can construct the following row.

| open      | single_blind | double_blind | masking      |
|-----------|--------------|--------------|--------------|
| No        | No           | Yes          | 2            |

It is clear that this row satisfies edit rule 1.
Moreover, all other edit rules are also satisfied by this row (check this).

Suppose now that we choose to adapt the value of attribute 'double_blind' to 'No' in the original row.
As a result, we can construct the following row.

| open      | single_blind | double_blind | masking      |
|-----------|--------------|--------------|--------------|
| Yes       | No           | No           | 2            |

This row also satisfies edit rule 1.
However, if we check this row against the other edit rules, we can see that this row does NOT satisfy edit rule 5: dom(open) $\times$ dom(single) $\times$ {No} $\times$ {2}.
This, again, gives a problem.

The reason that this happens is because edit rules 1 and 5 are not fully independent from each other.
Indeed, edit rule 1 states that *open* = 'Yes' and *double_blind* = 'Yes' cannot appear together in a row, and therefore, *open* = 'Yes' can only appear together with *double_blind* = 'No'.
Besides that, edit rule 5 states that *double_blind* = 'No' and *masking* = '2' cannot appear together in a row.
Both statements imply, thus, that *open* = 'Yes' may not appear together with *masking* = '2'.
We can now construct an *implied* edit rule, given these two edit rules, with the form {Yes} $\times$ dom(single) $\times$ dom(double) $\times$ {2}.

Formally, we can define the implication procedure as follows.

Consider a set $\mathcal{E}_c$ consisting of at least two edit rules (which we will call the *contributing set*) and an attribute $a_g$ (which we will call the *generator*).
An edit rule $E_1^* \times \cdots \times E_k^*$ consists of value sets $E_i^*$ for each attribute $a_i$, constructed by
* taking the union of the values of the rules in $\mathcal{E}_c$ for attribute $a_g$, and
* taking the intersection of the values of the rules in $\mathcal{E}_c$ for the other attributes.

If no value set $E_i^*$ is empty, $E_1^* \times \cdots \times E_k^*$ is called an *implied edit rule*.

Besides that, if attribute $a_g$ enters in each edit rule in $\mathcal{E}_c$, but it does not enter in the implied edit rule, we call the implied edit rule *essentially new*.

The set of explicitly given edit rules together with all essentially new rules is called the *complete set* of edit rules.
Fellegi & Holt have proven that the complete set of edit rules guarantees that each possible error can be localized correctly and the above situation cannot appear.

Try to understand the definition of an implied edit rule, an essentially new edit rule and the complete set before continuing the assignment.
Check whether the above example complies with this definition.

Now, it is clear that it is important to construct the complete set of edit rules in order to correctly identify attributes that are potentially in error.
Try to construct the complete set of edit rules manually, starting from the explicit set given above, by applying the Field Code Forest algorithm. 

In [ ]:
# Exercise: try to construct, manually, by means of the Field Code Forest algorithm, all essentially new edit rules and add them to the given set (the first one is already given in the example above)

edit_rules.append({'open' : 'Yes', 'masking' : '2'})
#edit_rules.append(...)

print("Edit rules: ")
for edit_rule in edit_rules:
    print(edit_rule)

In [ ]:
# Exercise: Persist, per row number, the edit rules that are failed by the corresponding row in the violations dictionary-object again.
# Use the algorithm that you have implemented earlier.

violations = get_violated_rows(dataset, edit_rules)
print("number of violated rows: {:d}".format(len(violations.keys())))
# 1224

### 3.3 Foutlokalisatie

Once the complete set of edit rules is generated and, for each row, the set of failing rules is identified, it is possible to correctly localize, for each failing row, the potential error.
This can be done by identifying the minimal covering set of attributes for each row.

A minimal covering set should meet two important properties.
* The set is covering: each failing edit rule has at least 1 entering attribute in the set.
This is straightforward, as we showed that we should adapt the value of at least 1 entering attribute in the edit rule to make the row satify this rule.
* The set is minimal: if you remove an attribute from this set, the covering property is not met anymore.
In other words, we want to adapt as few values as possible.

Suppose the dataset contains the following row.

| open      | single_blind | double_blind | masking      |
|-----------|--------------|--------------|--------------|
| No        | Yes          | Yes          | 0            |

In the original set of edit rules (so not the complete set), 3 failing edit rules are detected for this row, being

| Rule | open      | single_blind | double_blind | masking      |
|-------|-----------|--------------|--------------|--------------|
| 2     | dom(open) | Yes          | Yes          | dom(masking) |
| 6     | dom(open) | dom(single)  | Yes          | 0            |
| 7     | dom(open) | Yes          | dom(double)  | 0            |

The minimal sets of attributes covering these rules are {'single_blind', 'double_blind'}, {'single_blind', 'masking'} and {'double_blind', 'masking'}. 
Indeed, each failing rule has at least 1 entering attribute in these sets and the given sets are minimal (you cannot remove an attribute such that the set of attributes has still the covering property).
From these minimal covering sets, we choose one that we are going to use to fix the row.

In [ ]:
# Exercise: implement an algorithm that tests, for each failing row, sets consisting of 1 attribute, then sets consisting of 2 attributes, and so on, until a minimal covering set is found.
# Return, for each failing data object, one minimal covering set at random without taking into account the distributions.

import itertools

def get_minimal_covering_set(violated_rules):
    #...
    
minimal_covering_sets = {}
                
for row in violations:
    minimal_covering_sets[row] = get_minimal_covering_set(violations[row])     

**General note**

It is possible (as shown in previous examples) that multiple minimal covering sets exist for a row.
Strategies, most of them based on value distributions, exist to select (by probability) one minimal covering set for which the values should be adapted later.
This, however, is out of scope of this course, but do not hesitate to think about this.